In [1]:
# Inspiration :D
# https://gitlab.com/ryanlogsdon/pubsub-for-python/-/blob/main/subscriber.py
# https://www.youtube.com/watch?v=xOtrCmPjal8&ab_channel=D-I-Ry
import os
from google.cloud import pubsub_v1
import sys
import time
import pandas as pd
import http.client
import numpy as np
import datetime
from io import StringIO
import functools as ft

In [2]:
! python --version

Python 3.7.4


In [3]:
# Parameters section

location = "Hamburg"

## Create connection to PubSub

In [4]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [5]:
publisher_obs = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

publisher_forecast = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

topic_path_obs = 'projects/weather-based-forecasting-v2/topics/observations_topic'
topic_path_forecast = 'projects/weather-based-forecasting-v2/topics/forecasts_topic'

## Download the data

In [6]:
time.sleep(5)



conn = http.client.HTTPSConnection("visual-crossing-weather.p.rapidapi.com")

headers = {
    "X-RapidAPI-Key": "c7b48b7c4emsh8a3b688e71b9896p1be4d0jsnda2e5b27b486",
    "X-RapidAPI-Host": "visual-crossing-weather.p.rapidapi.com"
}


conn.request("GET", "/forecast?location={}&aggregateHours=1&shortColumnNames=0&unitGroup=metric&contentType=csv".format(location),
             headers=headers)

res = conn.getresponse()
data = res.read()
s = str(data, 'utf-8')
data_string = StringIO(s)
df = pd.read_csv(data_string)

In [7]:
df2 = df.copy()
df2.index = pd.to_datetime(df2["Date time"])
df2 = df2.asfreq(freq="1min")
df2 = df2.interpolate(method ='linear', limit_direction ='forward')

df2["Date time"] = df2.index

df3 = df2.iloc[:60*24*10, :]

In [8]:
df3

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2023-01-16 14:00:00,Hamburg,2023-01-16 14:00:00,53.5537,9.99183,"Hamburg, Deutschland",Hamburg,142.300000,4.000000,18.000000,100.000000,NaN,9.50,0.0,986.300000,0.0,0.0,78.700000,32.400000,1.900000,Overcast
2023-01-16 14:01:00,NaN,2023-01-16 14:01:00,53.5537,9.99183,NaN,NaN,142.436667,3.998333,18.048333,100.000000,NaN,9.50,0.0,986.278333,0.0,0.0,78.795000,32.471667,1.875000,NaN
2023-01-16 14:02:00,NaN,2023-01-16 14:02:00,53.5537,9.99183,NaN,NaN,142.573333,3.996667,18.096667,100.000000,NaN,9.50,0.0,986.256667,0.0,0.0,78.890000,32.543333,1.850000,NaN
2023-01-16 14:03:00,NaN,2023-01-16 14:03:00,53.5537,9.99183,NaN,NaN,142.710000,3.995000,18.145000,100.000000,NaN,9.50,0.0,986.235000,0.0,0.0,78.985000,32.615000,1.825000,NaN
2023-01-16 14:04:00,NaN,2023-01-16 14:04:00,53.5537,9.99183,NaN,NaN,142.846667,3.993333,18.193333,100.000000,NaN,9.50,0.0,986.213333,0.0,0.0,79.080000,32.686667,1.800000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-26 13:55:00,NaN,2023-01-26 13:55:00,53.5537,9.99183,NaN,NaN,68.083333,2.433333,14.525000,24.266667,NaN,4.40,0.0,1038.000000,0.0,0.0,77.333333,26.941667,-1.366667,NaN
2023-01-26 13:56:00,NaN,2023-01-26 13:56:00,53.5537,9.99183,NaN,NaN,68.006667,2.426667,14.520000,24.113333,NaN,4.48,0.0,1038.000000,0.0,0.0,77.366667,26.993333,-1.373333,NaN
2023-01-26 13:57:00,NaN,2023-01-26 13:57:00,53.5537,9.99183,NaN,NaN,67.930000,2.420000,14.515000,23.960000,NaN,4.56,0.0,1038.000000,0.0,0.0,77.400000,27.045000,-1.380000,NaN


In [9]:
now = datetime.datetime.now()
minutes = now.minute
df3_hour = df3.iloc[minutes-1:minutes+59, :]
df3_hour.head(5)

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2023-01-16 14:29:00,NaN,2023-01-16 14:29:00,53.5537,9.99183,NaN,NaN,146.263333,3.951667,19.401667,100.0,NaN,9.5,0.0,985.671667,0.0,0.0,81.455,34.478333,1.175,NaN
2023-01-16 14:30:00,NaN,2023-01-16 14:30:00,53.5537,9.99183,NaN,NaN,146.400000,3.950000,19.450000,100.0,NaN,9.5,0.0,985.650000,0.0,0.0,81.550,34.550000,1.150,NaN
2023-01-16 14:31:00,NaN,2023-01-16 14:31:00,53.5537,9.99183,NaN,NaN,146.536667,3.948333,19.498333,100.0,NaN,9.5,0.0,985.628333,0.0,0.0,81.645,34.621667,1.125,NaN
2023-01-16 14:32:00,NaN,2023-01-16 14:32:00,53.5537,9.99183,NaN,NaN,146.673333,3.946667,19.546667,100.0,NaN,9.5,0.0,985.606667,0.0,0.0,81.740,34.693333,1.100,NaN
2023-01-16 14:33:00,NaN,2023-01-16 14:33:00,53.5537,9.99183,NaN,NaN,146.810000,3.945000,19.595000,100.0,NaN,9.5,0.0,985.585000,0.0,0.0,81.835,34.765000,1.075,NaN


## Send Forecast in intervals to PubSub

In [10]:
df3_forecast = df3.iloc[minutes-1:, :]
df3_forecast.index = df3_forecast.index.astype(str)
df3_forecast

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2023-01-16 14:29:00,NaN,2023-01-16 14:29:00,53.5537,9.99183,NaN,NaN,146.263333,3.951667,19.401667,100.000000,NaN,9.50,0.0,985.671667,0.0,0.0,81.455000,34.478333,1.175000,NaN
2023-01-16 14:30:00,NaN,2023-01-16 14:30:00,53.5537,9.99183,NaN,NaN,146.400000,3.950000,19.450000,100.000000,NaN,9.50,0.0,985.650000,0.0,0.0,81.550000,34.550000,1.150000,NaN
2023-01-16 14:31:00,NaN,2023-01-16 14:31:00,53.5537,9.99183,NaN,NaN,146.536667,3.948333,19.498333,100.000000,NaN,9.50,0.0,985.628333,0.0,0.0,81.645000,34.621667,1.125000,NaN
2023-01-16 14:32:00,NaN,2023-01-16 14:32:00,53.5537,9.99183,NaN,NaN,146.673333,3.946667,19.546667,100.000000,NaN,9.50,0.0,985.606667,0.0,0.0,81.740000,34.693333,1.100000,NaN
2023-01-16 14:33:00,NaN,2023-01-16 14:33:00,53.5537,9.99183,NaN,NaN,146.810000,3.945000,19.595000,100.000000,NaN,9.50,0.0,985.585000,0.0,0.0,81.835000,34.765000,1.075000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-26 13:55:00,NaN,2023-01-26 13:55:00,53.5537,9.99183,NaN,NaN,68.083333,2.433333,14.525000,24.266667,NaN,4.40,0.0,1038.000000,0.0,0.0,77.333333,26.941667,-1.366667,NaN
2023-01-26 13:56:00,NaN,2023-01-26 13:56:00,53.5537,9.99183,NaN,NaN,68.006667,2.426667,14.520000,24.113333,NaN,4.48,0.0,1038.000000,0.0,0.0,77.366667,26.993333,-1.373333,NaN
2023-01-26 13:57:00,NaN,2023-01-26 13:57:00,53.5537,9.99183,NaN,NaN,67.930000,2.420000,14.515000,23.960000,NaN,4.56,0.0,1038.000000,0.0,0.0,77.400000,27.045000,-1.380000,NaN


In [ ]:
for i in range(60):
    
    #df3_forecast_window = str(df3_forecast.iloc[i:i+3*24*2,:].astype(str).to_dict("index"))
    df3_forecast_window = str(df3_forecast.iloc[i:i+60*24*2,:].astype(str).to_dict("index"))
    df3_forecast_window = df3_forecast_window.encode('utf-8')
        
    info = "Weather forecast for Hamburg, {}".format(df3_forecast.index[i])
    info = info.encode('utf-8')
    
    future = publisher_forecast.publish(topic_path_forecast, df3_forecast_window)
    print(f'published message id {future.result()}')
    time.sleep(60)
    #time.sleep(30)

published message id 6730312542375186
published message id 6730312729824447
published message id 6730312633556723
published message id 6730297110117290
published message id 6730315231151451
published message id 6730316392948213
published message id 6730315852155598
published message id 6730315076653653
published message id 6730318159559081
published message id 6730316393285678
published message id 6730320721844974
published message id 6730321596161881
published message id 6730318856892662
published message id 6730319282120034
published message id 6730322245236580
published message id 6730319282191806
published message id 6730322892887509
published message id 6730318454654741
published message id 6730324809910138
published message id 6730327718332396
published message id 6730330111034288
published message id 6730325015488536
published message id 6730327847240365
published message id 6730325015764207
published message id 6730328637048252
published message id 6730333506311992
published me